# Segmenting and Clustering Neighborhoods in Toronto
### Part 1. Data wrangling

Import all needed libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup 
import requests  

ImportError: No module named 'bs4'

Create url where we will scrap data

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Scrapping data using BeautifulSoup

In [ ]:
data = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")

We scrap all tables from the url. There are 16 tables.

In [ ]:
tables = soup.find_all('table')
len(tables)

We search our table of interest: the administrative list of city-designated neighbourhoods.

In [ ]:
for index,table in enumerate(tables):
    if ("M3A" in str(table)):
        table_index = index
print(table_index)

It is located in the position tables[10]. We look at this table

In [ ]:
print(tables[table_index].prettify())

We create the dataframe using Pandas and fill it with the data from the selected table. We don't import the last column (Map) because it is not necessary.

In [ ]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        if cell['Neighborhood'] == 'Not assigned':
            cell['Neighborhood'] = cell['Borough']
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [ ]:
df.head()

In [ ]:
df.shape